In [75]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [76]:
traindf_x = pd.read_csv("../data/train.csv", index_col="ID")
traindf_y = pd.read_csv("../data/train_label.csv")
testdf_x = pd.read_csv("../data/test.csv", index_col="ID")
testdf_y = pd.read_csv("../data/test_nolabel.csv")

adr = pd.read_csv("../data/TestAdr.csv")
is_canceled = pd.read_csv("../data/TestIsCanceled.csv")

In [84]:
testdf_x["adr"] = adr["adr"]
testdf_x["is_canceled"] = is_canceled["is_canceled"]

In [77]:
traindf_x.sample(2)

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
90126,90126,City Hotel,0,16,2017,March,12,23,0,1,...,No Deposit,NaN,NaN,0,Transient-Party,99.516653,0,0,Check-Out,2017-03-24
82701,82701,Resort Hotel,0,15,2017,February,5,3,0,2,...,No Deposit,240.0,NaN,0,Transient,68.566599,0,1,Check-Out,2017-02-05


In [78]:
def create_datetime(year: pd.Series, month: pd.Series, date: pd.Series) -> pd.Series:
    return pd.to_datetime(year.astype(str) + month + date.astype(str), format="%Y%B%d")

def year_month_to_date(df):
    df["date"] = create_datetime(df["arrival_date_year"], df["arrival_date_month"], df["arrival_date_day_of_month"])
    df.drop(["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"], 
        axis=1, inplace=True)
    return df

In [79]:
traindf_x = year_month_to_date(traindf_x)
testdf_x = year_month_to_date(testdf_x)

traindf_y["arrival_date"] = pd.to_datetime(traindf_y["arrival_date"])
testdf_y["arrival_date"] = pd.to_datetime(testdf_y["arrival_date"])

In [80]:
traindf_x.sample(2)

,ID,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,date
8550,8550,Resort Hotel,0,119,37,2,4,2,0.0,0,...,240.0,NaN,0,Transient,80.985945,0,1,Check-Out,2015-09-18,2015-09-12
75449,75449,City Hotel,0,16,50,2,0,2,1.0,0,...,9.0,NaN,0,Transient,145.635763,0,1,Check-Out,2016-12-06,2016-12-04


In [81]:
def stay_nights(df):
    return df["stays_in_weekend_nights"] + df["stays_in_week_nights"]

traindf_x["stay_nights"] = stay_nights(traindf_x)
testdf_x["stay_nights"] = stay_nights(testdf_x)

traindf_x = traindf_x[traindf_x["stay_nights"] != 0]
testdf_x = testdf_x[testdf_x["stay_nights"] != 0]

In [82]:
def explode_date(df):
    df['date'] = [pd.date_range(s, periods=e, freq='d') for s, e in
              zip(df['date'], df['stay_nights'])]
    df = df.explode("date")
    return df

traindf_x = explode_date(traindf_x)
testdf_x = explode_date(testdf_x)

In [85]:
def true_adr(df):
    return df["adr"] * (1 - df["is_canceled"])

traindf_x["adr"] = true_adr(traindf_x)
testdf_x["adr"] = true_adr(testdf_x)

In [110]:
traindf = traindf_y.merge(traindf_x.groupby("date").agg("sum")["adr"].reset_index(level=0), how="left",
         left_on="arrival_date", right_on="date")
testdf = testdf_y.merge(testdf_x.groupby("date").agg("sum")["adr"].reset_index(level=0), how="left",
         left_on="arrival_date", right_on="date")

In [159]:
testdf.sample(2)

,arrival_date,date,adr
55,2017-05-26,2017-05-26,20593.350426
3,2017-04-04,2017-04-04,17299.821347


In [126]:
traindf = traindf.loc[20:]

In [156]:
new_train_x = traindf[["adr"]]
new_train_y = traindf["label"].replace([6, 7, 8, 9], 6).astype("int64")

In [172]:
#pd.concat([testdf_y, answer_label], axis=1).to_csv("../data/answer.csv", header=["arrival_date", "label"], index=False)